<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/AlBerto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlBERTo Hate Speech Classifier 

In [4]:
!pip install ekphrasis
!pip install tensorflow-gpu==1.15.0

import datetime
import sys
import warnings
warnings.filterwarnings("ignore")

#for code working
import tensorflow as tf
import re
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import numpy as np
import pandas as pd 

#Prepare and import BERT modules
import subprocess
subprocess.call(["git", "clone", "https://github.com/google-research/bert","bert_repo"])

if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import tokenization

In [2]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


## Load the dataset 

In [5]:
# directory name 
input_dir = '/content/drive/My Drive/HLT/clean_dataset_training/' 
AlBERTo_path = '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/'

# Spec
pd.set_option("display.max_colwidth", None)

In [38]:
tsv_file = open(input_dir+"training_dataset.csv")

dataset = pd.read_csv(tsv_file,sep=',')

# copy of tweets text 
tweets = dataset['text']  

## Configure AlBERTo classes 

In [7]:
text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=[ 'url' , 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'] ,
    # terms that will be annotated
    annotate={"hashtag"} ,
    fix_html=True ,  # fix HTML tokens

    unpack_hashtags=True ,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts = [ emoticons ]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt
Reading english - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_2grams.txt
Reading english - 1grams ...


In [8]:
dataset_sentences = dataset['text']
dataset_labels = dataset['hs']

In [9]:
i = 0 
examples = []
for s in dataset_sentences: 
  examples.append([dataset_labels[i], s])
  i = i+1

examples = np.array(examples)

In [10]:
'''
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create InputExample's using the constructor provided in the BERT library.

    text_a is the text we want to classify, which in this case, is the Request field in our Dataframe.
    text_b is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave text_b blank.
    label is the label for our example, i.e. True, False

'''
# guid = Globally unique ID for bookkeeping, unused in this example
f = lambda x: InputExample(guid=None, text_a = x[1], text_b = None, label = int(x[0]))

examples = map(f, examples)
examples = list(examples)

# Tokenizer 

In [11]:
VOCAB_FILE = '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/vocab.txt'

#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(VOCAB_FILE, do_lower_case=True)

# Load Pre-Trained Model 

In [12]:
#model = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

In [64]:
OUTPUT_DIR = '/content/drive/My Drive/HLT/output'
#Inizialize AlBERTo
INIT_CHECKPOINT = AlBERTo_path + 'alberto_model.ckpt'
#SET THE PARAMETERS
PREDICT_BATCH_SIZE = 1
MAX_SEQ_LENGTH = 128
LEARNING_RATE = 2e-5
label_list = [0, 1]
BERT_CONFIG= modeling.BertConfig.from_json_file(AlBERTo_path + "config.json")

In [65]:
model_fn = model_fn_builder(
  bert_config=BERT_CONFIG,
  num_labels=2,
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=0,
  num_warmup_steps=0,
  use_tpu=False,
  use_one_hot_embeddings=True
)

# Prepare predictions

In [66]:
# Setup sentences
input_features = convert_examples_to_features(
      examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=input_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

INFO:tensorflow:Writing example 0 of 6837
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] e terrorismo anche questo per mettere in uno stato di soggezione le persone e render ##le innocue mentre qualcuno [SEP]
INFO:tensorflow:input_ids: 2 13 4923 23 79 22 605 24 153 184 12 49535 40 234 13 20897 1041 90954 408 271 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [67]:
len(input_features)

6837

## Setup run configuration 

In [68]:
def get_run_config():
  return tf.contrib.tpu.RunConfig(
    model_dir=AlBERTo_path)

In [69]:
estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=False,
  model_fn=model_fn,
  predict_batch_size=PREDICT_BATCH_SIZE,
  config=get_run_config()
)

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f16856103d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_p

In [ ]:
predictions = estimator.predict(predict_input_fn)

for example, prediction in zip(tweets,predictions):
    print('\t prediction:%s \t text_a: %s' % ( np.argmax(prediction['probabilities']),str(example) ) )
#print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
#for example, prediction in zip(sentences,predictions):
 #   print('\t prediction:%s \t text_a: %s' % ( np.argmax(prediction['probabilities']),str(example) ) )

INFO:tensorflow:Could not find trained model in model_dir: /content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)

INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (128000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), 